https://docs.smith.langchain.com/cookbook/fine-tuning-examples/export-to-openai

In [3]:
import os

os.environ.get("OPENAI_API_KEY")
os.environ.get("LANGCHAIN_API_KEY")
print()

In [4]:
# open ai api key, langchain(langsmith) apikey 등록

os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "default"  # if not specified, defaults to "default"

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-47433b0f-7af9-4225-8d59-674b8565f8b8-0')

In [6]:
from langsmith import Client

client = Client()

Query runs

In [7]:
import datetime

project_name = "default"
run_type = "llm"
end_time = datetime.datetime.now()

runs = client.list_runs(
    project_name=project_name,
    run_type=run_type,
    error=False,
)

filter by feedback

In [8]:
schemalist = sorted(dir(schema), reverse=True)
print(schemalist)

NameError: name 'schema' is not defined

In [9]:
from langchain import chains, chat_models, prompts, schema, callbacks

chain = prompts.ChatPromptTemplate.from_template("Tell a joke for :\n{input}") | chat_models.ChatOpenAI(tags=['my-openai-run']) | schema.StrOutputParser()

with callbacks.collect_runs() as cb :
    chain.invoke({"input" : "foo"})
    
    # Assume feedback is logged
    run = cb.traced_runs[0]
    client.create_feedback(run.id, key = "user_click", score = 1)

/Users/parksunyoung/anaconda3/envs/llm/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
project_name = "default"
end_time = datetime.datetime.now()

runs = client.list_runs(
    project_name=project_name,
    execution_order=1,
    filter='and(eq(feedback_key, "user_click"), eq(feedback_score, 1))',
    error=False,
)

In [11]:
llm_runs = []
for run in runs :
    llm_run = next(client.list_runs(project_name=project_name, run_type="llm", parent_run_id=run.id))
    llm_runs.append(llm_run)

llm_runs[0].tags

['my-openai-run', 'seq:step:2']